# Sentiment Analysis

Codice scalabile per calcolare il sentiment dei commenti.

Si implementa un sistema di **logging** per tracciare l'esecuzione del codice e facilitare il debugging.

In [1]:
# Importazioni
import pandas as pd
from multiprocessing import Pool
import logging
import yaml

In [2]:
# Configurazione del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filemode='w')

In [3]:
# Funzione per caricare i dati
def load_data(file_path):
    logging.info(f'Caricamento dati da {file_path}')
    return pd.read_csv(file_path)

## Funzioni

### Salvare i risultati

In [4]:
import os

# Funzione per salvare i risultati 
def save_results(df, file_path):
    # Verifica se il file esiste
    if not os.path.isfile(file_path):
        df.to_csv(file_path, index=False, mode='w')
    else:
        df.to_csv(file_path, index=False, mode='a', header=False)
    logging.info(f'Salvati i risultati parziali in {file_path}')

### Preprocessing

Esiste un tokenizzatore già addestrato su dati testuali quali tweet o commenti social. Può essere utilizzato anche per il italiano, anche se è stato originariamente progettato per testi in stile tweet in inglese. Tuttavia, poiché i tweet possono contenere caratteristiche multilingue simili (ad esempio emoticon, hashtag, abbreviazioni), TweetTokenizer può funzionare ragionevolmente bene per la tokenizzazione di testi in altre lingue, inclusa l'italiano.

In [5]:
from nltk.tokenize import TweetTokenizer
import string
import spacy
from nltk.corpus import stopwords
!python -m spacy download it_core_news_sm

2024-06-26 22:22:26,483 - WARNING - From c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
      --------------------------------------- 0.2/13.0 MB 3.7 MB/s eta 0:00:04
      --------------------------------------- 0.2/13.0 MB 2.8 MB/s eta 0:00:05
     - -------------------------------------- 0.4/13.0 MB 2.5 MB/s eta 0:00:05
     - -------------------------------------- 0.5/13.0 MB 2.7 MB/s eta 0:00:05
     - -------------------------------------- 0.6/13.0 MB 2.7 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/13.0 MB 3.0 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/13.0 MB 3.0 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/13.0 MB 2.9 MB/s eta 0:00:05
     --- ------------------------------------ 1.3/13.0 MB 3.0 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/13.0 MB 3.1 MB/s eta 0:00:04
     ---- ----------------------------------- 1.6/13.0 MB 3.1 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/13.0 MB 3

2024-06-26 22:22:30.991868: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.



In [6]:
nlp = spacy.load("it_core_news_sm") # Carichiamo il modello italiano di spaCy

# Funzione per lemmatizzare una lista di token
def lemmatize_tokens(tokens):
    doc = nlp(" ".join(tokens))  # Converto i token in una stringa e processo con spaCy
    return [token.lemma_ for token in doc]  # Estraggo i lemmi

In [7]:
# Si utilizza una libreria specifica per la tokenizzazione multilingue come spaCy, 
# che ha modelli pre-addestrati per molte lingue, tra cui l'italiano.

tokening = TweetTokenizer() # Tokenizzazione per i tweet
stop = stopwords.words('italian')   # Carichiamo le stopwords italiane     
punctuation = string.punctuation    # Carichiamo la punteggiatura

# Tengo le parole non e più perché potrebbero essere importanti per il sentimento
stop = [item for item in stop if item not in ["non", "più"]]

# Funzione per eseguire il preprocessing dei dati
def preprocess_data(df):
    
    # Quando abbiamo a che fare con il testo dei social media, di solito vogliamo identificare URL, 
    # hashtag e emoticon come oggetti separati e non tokenizzarli in singoli caratteri:
    comments_tokenized = df.apply(TweetTokenizer().tokenize)

    # Rimuoviamo le stopword
    comments_tokenized_stop = comments_tokenized.apply(lambda x: [item for item in x if item not in stop])
    
    # Rimuoviamo la punteggiatura
    comments_tokenized_stop_punct = comments_tokenized_stop.apply(lambda x: [item for item in x if item not in punctuation])

    # Lemmatizzazione
    comments_tokenized_new_stem = comments_tokenized_stop_punct.apply(lemmatize_tokens)

    # Unisco i token lemmatizzati per formare le frasi preprocessate
    preprocessed_comments = comments_tokenized_new_stem.apply(lambda x: " ".join(x))

    return preprocessed_comments

### NRC Emotion Lexicon

L'**NRC Emotion Lexicon** ha annotazioni influenti per le parole inglesi. Nonostante alcune differenze culturali, è stato dimostrato che la maggior parte delle norme affettive sono stabili attraverso le lingue. Pertanto, forniamo versioni del lessico in oltre 100 lingue traducendo i termini inglesi utilizzando Google Translate (agosto 2022).

Il lessico è quindi disponibile per l'inglese ma anche per numerose altre lingue, tra cui l'italiano!

In [8]:
import pandas as pd

Emotion_Lexicon = "../resources/NRC-Emotion-Lexicon.xlsx"
lexicon_df = pd.read_excel(Emotion_Lexicon, engine="openpyxl")

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
lexicon = {}
for word, pos, neg, ant, ang, dis, fea, joy, sad, sur, tru in zip(lexicon_df["Italian Translation (Google Translate)"], lexicon_df["Positive"], lexicon_df["Negative"], lexicon_df["Anticipation"], lexicon_df["Anger"], lexicon_df["Disgust"], lexicon_df["Fear"], lexicon_df["Joy"], lexicon_df["Sadness"], lexicon_df["Surprise"], lexicon_df["Trust"]): 
    value = []
    if pos:
        value.append("positive")
    if neg:
        value.append("negative")
    if ant:
        value.append("anticipation")
    if ang:
        value.append("anger")
    if dis:
        value.append("disgust")
    if fea:
        value.append("fear")
    if joy:
        value.append("joy")
    if sad:
        value.append("sadness")
    if sur:
        value.append("surprise")
    if tru:
        value.append("trust")
    lexicon[str(word).lower()] = value #lower case

In [10]:
# Funzione che calcola il sentiment con NRC-Emotion-Lexicon
def nrc_score(df):
    # Inizializzo i contatori per i sentimenti
    positive_count = 0
    negative_count = 0
    
    # Inizializzo un dizionario per contare le emozioni
    emotion_count = {
        "anticipation": 0,
        "anger": 0,
        "disgust": 0,
        "fear": 0,
        "joy": 0,
        "sadness": 0,
        "surprise": 0,
        "trust": 0
    }
    
    # Analizzo ogni parola nella frase
    for word in df.split():  # La funzione split divide la stringa dagli spazi
        word = word.lower()
        if word in lexicon:
            values = lexicon[word]
            # Conto i sentimenti
            if "positive" in values:
                positive_count += 1
            if "negative" in values:
                negative_count += 1
            # Conta le emozioni
            for emotion in emotion_count.keys():
                if emotion in values:
                    emotion_count[emotion] += 1

    # Determino il sentimento prevalente
    if positive_count > negative_count:
        predominant_sentiment = "positive"
    elif negative_count > positive_count:
        predominant_sentiment = "negative"
    else:
        predominant_sentiment = "neutral"  

    # Determina l'emozione prevalente
    if all(value == 0 for value in emotion_count.values()):
        predominant_emotion = "none"  # Nessuna emozione prevalente
    else:
        predominant_emotion = max(emotion_count, key=emotion_count.get)
    
    # Restituisco una lista con i due valori prevalenti
    return [predominant_sentiment, predominant_emotion]

### Italian BERT Sentiment model

https://huggingface.co/neuraly/bert-base-italian-cased-sentiment

Questo modello esegue l'analisi del sentiment sulle frasi italiane. È stato addestrato a partire da un'istanza di bert-base-italian-cased, e messo a punto su un dataset italiano di tweets, raggiungendo su quest'ultimo l'82% di precisione.

In [11]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm



def bert_score(df):

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained("neuraly/bert-base-italian-cased-sentiment")

    # Load the model, use .cuda() to load it on the GPU
    bert_model = AutoModelForSequenceClassification.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
    bert_model.eval()  # Imposta il modello in modalità valutazione

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        sentence = row.comment_body

        input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)

        # Create tensor, use .cuda() to transfer the tensor to GPU if available
        tensor = torch.tensor(input_ids).long().unsqueeze(0)
        if torch.cuda.is_available():
            tensor = tensor.cuda()
            bert_model.cuda()

        # Call the model and get the logits
        with torch.no_grad():
            logits = bert_model(tensor)
        
        # Remove the fake batch dimension
        logits = logits.logits.squeeze(0)

        # Apply softmax to get probabilities
        proba = nn.functional.softmax(logits, dim=0)

        # Unpack the tensor to obtain negative, neutral and positive probabilities
        negative, neutral, positive = proba

        # Assign probabilities to DataFrame
        df.loc[index, 'negative'] = proba[0].item()
        df.loc[index, 'neutral'] = proba[1].item()
        df.loc[index, 'positive'] = proba[2].item()

    # Determine the sentiment label based on maximum probability
    df['BERT'] = df[['negative', 'positive', 'neutral']].idxmax(axis=1)

    db = df[['comment_id', 'post_id', 'debate_group', 'comment_user_name', 'comment_body', 'comment_score', 'preprocessed_comment', 'negative', 'neutral', 'positive', 'BERT']]

    # Salvo i risultati in un file
    save_results(db, '../results/BERT_results.csv')

    # Rimuovo le colonne di probabilità
    df.drop(['negative', 'neutral', 'positive'], axis=1, inplace=True)

### FEEL-IT: Emotion and Sentiment Classification for the Italian Language

https://huggingface.co/MilaNLProc/feel-it-italian-sentiment

Utilizzo un altro classificatore di sentiment, dotato di un corpus di riferimento di post Twitter italiani annotati con quattro emozioni fondamentali: rabbia, paura, gioia, tristezza. Comprimendoli, possiamo anche fare l’analisi del sentiment. 

PAPER: https://aclanthology.org/2021.wassa-1.8.pdf

In [12]:
from feel_it import SentimentClassifier
from feel_it import EmotionClassifier
emotion_classifier = EmotionClassifier()
sentiment_classifier = SentimentClassifier()

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
def feel_score(df):
    feel_it_scores = []
    for sentence in df['preprocessed_comment']:
        feel_it_scores.append(sentiment_classifier.predict([sentence])[0])
    df['FEEL_IT'] = feel_it_scores
    db = df[['comment_id', 'post_id', 'debate_group', 'comment_user_name', 'comment_body', 'comment_score', 'preprocessed_comment', 'FEEL_IT']]

    # Salvo i risultati in un file CSV
    save_results(db, "../results/FEEL_IT_results.csv")


In [14]:
def emotion_score(df):
    emotion_scores = []
    for sentence in df['preprocessed_comment']:
        emotion_scores.append(emotion_classifier.predict([sentence])[0])
    df['EMOTION'] = emotion_scores
    db = df[['comment_id', 'post_id', 'debate_group', 'comment_user_name', 'comment_body', 'comment_score', 'preprocessed_comment', 'EMOTION']]
    save_results(db, "../results/EMOTION_results.csv")


### MULTILINGUAL: bert-base-multilingual-uncased-sentiment 

https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment 

Si tratta di un modello bert-base-multilingue-uncased ottimizzato per l'analisi del sentiment sulle recensioni dei prodotti in sei lingue: inglese, olandese, tedesco, francese, spagnolo e **italiano**. Prevede il sentiment della recensione sotto forma di numero di stelle (tra 1 e 5).

In [15]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment").to("cuda")

In [16]:
from tqdm import tqdm

def multilingual_score(df):
    # Crea colonne per memorizzare le probabilità
    df['1'] = 0
    df['2'] = 0
    df['3'] = 0
    df['4'] = 0
    df['5'] = 0

     # Itera attraverso ogni riga del DataFrame
    for index, row in tqdm(df.iterrows()):
        sentence = row.preprocessed_comment
            
        input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)
            
        # Create tensor, use .cuda() to transfer the tensor to GPU
        tensor = torch.tensor(input_ids).long()
        # Fake batch dimension
        tensor = tensor.unsqueeze(0).to("cuda")

        # Call the model and get the logits
        logits = model(tensor)
        # Remove the fake batch dimension
        logits = logits.logits.squeeze(0)

        # The model was trained with a Log Likelyhood + Softmax combined loss, hence to extract probabilities we need a softmax on top of the logits tensor
        proba = nn.functional.softmax(logits, dim=0)

        # Aggiungi le probabilità alle colonne appropriate
        proba = proba.to("cpu")
        df.loc[index, '1'] = proba[0].item()
        df.loc[index, '2'] = proba[1].item()
        df.loc[index, '3'] = proba[2].item()
        df.loc[index, '4'] = proba[3].item()
        df.loc[index, '5'] = proba[4].item()

    # Calcola il sentimento prevalente
    df['MULTILINGUAL'] = df[['1', '2', '3', '4', '5']].idxmax(axis=1)
    db = df[['comment_id', 'post_id', 'debate_group', 'comment_user_name', 'comment_body', 'comment_score', 'preprocessed_comment', '1', '2', '3', '4', '5', 'MULTILINGUAL']]
    df.drop(columns=['1', '2', '3', '4', '5'], inplace=True)

    # Tentare di convertire i valori della colonna 'MULTILINGUAL' in numerici, ignorando gli errori
    df['sentiment_pred_numeric'] = pd.to_numeric(df['MULTILINGUAL'], errors='coerce')

    # Applicare le trasformazioni solo ai valori numerici
    df.loc[df['sentiment_pred_numeric'] >= 4, 'MULTILINGUAL'] = 'positive'
    df.loc[df['sentiment_pred_numeric'] <= 2, 'MULTILINGUAL'] = 'negative'
    df.loc[df['sentiment_pred_numeric'] == 3, 'MULTILINGUAL'] = 'neutral'

    # Rimuovere la colonna temporanea utilizzata per la conversione
    df.drop(columns=['sentiment_pred_numeric'], inplace=True)
    
    save_results(db, "../results/MULTILINGUAL_results.csv")

### Predirre sentiment ed emotion con i modelli

In [17]:
# Funzione per predire il sentiment usando un modello specifico
def predict(model_name, df):
    # Implementa la logica di predizione per ciascun modello
    if(model_name == 'NRC'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm
        df['NRC'] = df["preprocessed_comment"].apply(nrc_score)
        save_results(df, '../results/NRC_results.csv')
    elif(model_name == 'BERT'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm
        df = bert_score(df)
    elif(model_name == 'FEEL'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm
        df = feel_score(df)
    elif(model_name == 'MULTILINGUAL'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm
        df = multilingual_score(df)
    elif(model_name == 'EMOTION'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm        
        df = emotion_score(df)
    return df

### Etichettatura

Si procede etichettando il dataset risultante per maggioranza: si comparano i risultati dei tre modelli più utilizzati e, il sentimento che prevale, sarà quello associato al dato commento. Quando ho tre sentimenti differenti, imposto "neutral".

In [18]:
from collections import Counter

def majority_vote(row):
    # Ottieni le predizioni per la riga corrente
    predictions = [row['BERT'], row['FEEL_IT'], row['MULTILINGUAL']]
    
    # Conta le occorrenze di ogni predizione
    vote_counts = Counter(predictions)
    
    # Trova la predizione con il maggior numero di voti
    majority_vote = vote_counts.most_common(1)[0][0]
    
    return majority_vote

def apply_majority_vote(df):
    # Applica la funzione majority_vote a ogni riga del DataFrame
    df['majority_vote'] = df.apply(majority_vote, axis=1)
    return df

### Calcolo di sentiment ed emotion di una discussione

In [113]:
def sentiment_and_emotion_discussion(df):
    # Raggruppare per emozione e calcolare il punteggio totale ponderato per ogni emozione
    sentiment_scores = df.groupby('majority_vote')['comment_score'].sum()
    emotion_scores = df.groupby('EMOTION')['comment_score'].sum()


    # Determinare l'emozione prevalente per ogni gruppo di discussione
    prevalent_sentiment = df.groupby('debate_group').apply(lambda group: sentiment_scores.idxmax())
    prevalent_emotion = df.groupby('debate_group').apply(lambda group: emotion_scores.idxmax())


 # Creare il dataframe discussion con le informazioni desiderate
    discussion = pd.DataFrame({
        'post_id': df['post_id'].iloc[0],
        'debate_group': prevalent_emotion.index,
        'sentiment': prevalent_sentiment.values,
        'emotion': prevalent_emotion.values
    })

    # Salvare i risultati in un file CSV
    save_results(discussion, '../results/discussion_sentiment_and_emotion.csv')
  

### Calcolo pesato del sentiment di un post 

In [139]:
import numpy as np


def post_sentiment(df, relevance):
    # Mappa le etichette 'positive', 'negative', 'neutral' a 1, -1, 0 rispettivamente
    sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}
    df['sentiment_numeric'] = df['sentiment'].map(sentiment_mapping)
    
    # Verifica che il numero di righe nel DataFrame df corrisponda alla lunghezza di relevance
    if len(df) != len(relevance):
        raise ValueError("La lunghezza di relevance deve corrispondere al numero di righe nel DataFrame df.")
    
    # Converti relevance in un array NumPy per la moltiplicazione
    relevance_array = np.array(relevance)
    
    # Calcola l'importanza pesata del sentiment numerico per ciascuna discussione
    weighted_sentiments = df['sentiment_numeric'] * relevance_array
    
    # Calcola il sentiment totale del post sommando tutte le importanze pesate
    post_sentiment = weighted_sentiments.sum()
    
    return post_sentiment

## Main

In [19]:
# Caricamento della configurazione
with open("../resources/config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [20]:
 # Caricamento dei dati (Si utilizza il dataset generato al termine dell'analisi dei grafi, 
 # composto esclusivamente dai commenti che hanno contribuito attivamente ai dibattiti di ciascun post. 
 # Lo si vuole analizzare per capire se tali commenti abbiano un sentiment positivo o negativo.)
df = load_data(config['data']['input_path'])

2024-06-26 22:23:14,477 - INFO - Caricamento dati da ../debate_dataset.csv


In [21]:
# Creo una lista di id_post unici
id_posts = df['post_id'].unique()

print(id_posts)
print("Numero di post unici: ", len(id_posts))


# Creo una lista di id_post unici
debate_groups = df['debate_group'].unique()

['197vo6o' '19aeo2k' '1b6cg4q' '1cwqkqe' '1bulhj9' '1d5h5h6' '17z2hci'
 '17lese9' '10v8sey']
Numero di post unici:  9


In [ ]:
# Filtro il dataset in modo tale da avere solo i commenti relativi ad un post

# Itero sugli id dei post
for id_post in id_posts:
    # Filtro il dataframe per l'id del post corrente
    df_filtered_post = df[df['post_id'] == id_post]
    for debate_group in df_filtered_post['debate_group'].unique():
        # Filtro il dataframe per il gruppo di dibattito corrente
        df_filtered_group = df_filtered_post[df_filtered_post['debate_group'] == debate_group]
        
        # Stampo il numero di commenti per il post corrente e gruppo di dibattito
        print(f"Numero di commenti per il post {id_post} che hanno partecipato al debate group {debate_group} : {len(df_filtered_group)}")
        
        # Isolo le uniche colonne che mi interessano
        df_fil = df_filtered_group[['comment_id', 'comment_body']].copy()
        
        # Estraggo solo i commenti
        comments = df_fil['comment_body']
        
        # Preprocesso i commenti
        preprocessed_comments = preprocess_data(comments)
        
        df_filtered_group["preprocessed_comment"] = preprocessed_comments
        
        # Utilizzo un indice booleano per assegnare direttamente i valori nel DataFrame originale
        df.loc[df_filtered_group.index, 'preprocessed_comment'] = preprocessed_comments
        
        # Salvo i risultati parziali
        save_results(df.loc[df_filtered_group.index, ['comment_id', 'comment_body', 'preprocessed_comment']], "../results/preprocessed_comments.csv")

        
        # Predizioni con i modelli
        models = config['models']
        for mod in models:
            predict(mod, df_filtered_group)
        
        # Salvo i risultati parziali
        save_results(df_filtered_group, "../results/predictions.csv")

        # Etichetto il sentimento prevalente
        apply_majority_vote(df_filtered_group)

        # Salvo i risultati parziali
        save_results(df_filtered_group, "../results/majority_vote.csv")

## Analisi


A questo punto si analizzano i risultati ottenuti.

In [107]:
# Voglio analizzare il sentiment di ciascuna discussione e salvarlo in un file
df = load_data("../results/majority_vote.csv")

# Filtro il dataset dei risultati ottenuto prima per avere solo le colonne che mi interessano
df = df[['comment_id', 'post_id', 'debate_group', 'comment_body', 'comment_score', 'EMOTION', 'majority_vote']] 

2024-06-27 00:04:49,993 - INFO - Caricamento dati da ../results/majority_vote.csv


In [ ]:
debate_groups = df['debate_group'].unique()

for debate_group in debate_groups:
    # Filtro il dataframe per il gruppo di dibattito corrente
    df_filtered_group = df[df['debate_group'] == debate_group]
    
    # Applico la funzione sentiment_discussione per ottenere il sentiment prevalente per ciascuna discussione
    sentiment_and_emotion_discussion(df_filtered_group)

Voglio conoscere il sentiment generale del post, ma lo voglio pesato sulla base dell'importanza di ciascuna discussione:

In [109]:
sentiment_post = load_data("../results/discussion_sentiment_and_emotion.csv")

2024-06-27 00:04:56,032 - INFO - Caricamento dati da ../results/discussion_sentiment_and_emotion.csv


In [140]:
import random

id_posts = sentiment_post['post_id'].unique()


sentiment = []

for id_post in id_posts:
    # Filtro il dataframe per il gruppo di dibattito corrente
    df_filtered_post = sentiment_post[sentiment_post['post_id'] == id_post]
    # per ora randomica
    relevance = np.random.rand(sentiment_post.shape[0])
    # Applico la funzione sentiment_discussione per ottenere il sentiment prevalente per ciascuna discussione
    sentiment.append(post_sentiment(sentiment_post, relevance))

In [141]:
sentiment

[-13.102443197410024,
 -10.748610443179693,
 -9.780502262993078,
 -12.963043719922785,
 -11.204830894620825,
 -12.619593501385078,
 -11.37667778844152,
 -13.159845578664278,
 -8.391018789333012]